In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import library
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import tree

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')

# **EDA**

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isna().sum()

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df['ST_Slope'].unique()

In [ ]:
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['ChestPainType'] = le.fit_transform(df['ChestPainType'])
df['RestingECG'] = le.fit_transform(df['RestingECG'])
df['ExerciseAngina'] = le.fit_transform(df['ExerciseAngina'])
df['ST_Slope'] = le.fit_transform(df['ST_Slope'])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
# declare features and target
X = df.drop(columns='HeartDisease')
y = df['HeartDisease']

In [ ]:
X.info()

In [ ]:
y.info()

In [ ]:
len(df)

In [ ]:
# lihat jumlah data
print(f'Count of X: {X.shape}\ncount of y: {y.shape}')

In [ ]:
c0 = df[df.HeartDisease==0]
c1 = df[df.HeartDisease==1]
print(f'count of data with label 0: {len(c0)}\ncount of data with label 1: {len(c1)}')

In [ ]:
# see proportion of labels
sns.countplot(x='HeartDisease', data=df, edgecolor='black')

# 410 tidak menderita sakit jantung
# 508 menderita sakit jantung

In [ ]:
# plot comparison age by death event
sns.histplot(c0.Age,kde=False,color="blue", label="Age for Diagnosis 0")
sns.histplot(c1.Age,kde=False,color = "red", label = "Age for Diagnosis 1")
plt.title("Histograms for Age by Death Event")
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get the number of features and calculate the number of subplots needed
num_features = len(X.columns)
num_rows = (num_features - 1) // 2 + 1
num_cols = min(num_features, 2)

# Create the figure and axis
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(10, 8))

# Flatten the axes if necessary
axes = axes.flatten()

# Loop through each feature and plot the boxplot
for i, feature in enumerate(X.columns):
    ax = axes[i]
    
    sns.boxplot(data=X, x=feature, ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel(feature.capitalize())
    ax.set_title(f'Boxplot of {feature.capitalize()}')

# Hide any unused subplots
for j in range(i+1, num_rows*num_cols):
    fig.delaxes(axes[j])

# Adjust the spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Check Outlier Before Drop Outlier

df.hist(bins = 30, figsize=(20, 15), color = 'red');

In [ ]:
print("Original DataFrame:")
pd.DataFrame(X.describe()).T

In [ ]:
# visualize to see distribution of the data
sns.set_style(style='darkgrid')
sns.pairplot(df)

In [ ]:
plt.figure(figsize=(10, 5))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
# # Convert input data to NumPy arrays
# X = np.array(X)
# y = np.array(y)

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=11)

# **Non Pruning Decision Tree**

In [ ]:
# declare model dtc
dtc = DTC(random_state=11,
          criterion='entropy')

In [ ]:
# fitting the train data into model
dtc.fit(X_train, y_train)

In [ ]:
# do a prediction using test data
npr_pred = dtc.predict(X_test)
acc_dtc = accuracy_score(y_test, npr_pred)
acc_dtc

In [ ]:
# make a classification report
print(classification_report(y_test, npr_pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_test, npr_pred)

ax = sns.heatmap(cm, annot=True, cmap="Reds")

ax.set_xlabel('Prediksi')
ax.set_ylabel('Aktual')
plt.title('Confussion Matrix of Decision Tree CART')
plt.show()

In [ ]:
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, npr_pred)
print(cf_matrix)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
plt.figure(figsize=(15,10))
tree.plot_tree(dtc,filled=True)

# **Cost Complexity Decision Tree**

In [ ]:
clf = DTC(random_state=11, 
          criterion='entropy')

In [ ]:
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DTC(random_state=11,
                             ccp_alpha=ccp_alpha, )
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
# in alpha -+0.3 accuracy model has incrased, use alpha 0.015 to pruning parameter (ccp_alpha)
# when more the alpha increases, the more the decision tree is cut
clf = DTC(random_state=11,
          ccp_alpha=0.015, 
          criterion='entropy')
clf.fit(X_train,y_train)

In [ ]:
# do a prediction and check accuracy score
pr_pred=clf.predict(X_test)
acc_prdtc = accuracy_score(y_test, pr_pred)
acc_prdtc

In [ ]:
# print classification report
print(classification_report(y_test, pr_pred))

In [ ]:
# visualize the pruned tree
plt.figure(figsize=(15,10))
tree.plot_tree(clf,filled=True)

# **Bagging Classifier**

In [ ]:
base_estimator = DTC()
bagging = BaggingClassifier(base_estimator)

param_grid = {
    'base_estimator__max_depth': [None, 3, 5],
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'random_state': [42]
}

grid_search = GridSearchCV(bagging, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

In [ ]:
# declare model bagging classifier with best parameter from hyperparameter tuning
bcl = BaggingClassifier(estimator=DTC(criterion='entropy',random_state=11), 
                        n_estimators=100, max_samples=0.9,max_features=0.7, random_state=11, n_jobs=-1)

In [ ]:
# fitting the model
bcl.fit(X_train, y_train)

In [ ]:
# do prediction with data test
bcl_pred = bcl.predict(X_test)

In [ ]:
# write accuracy score
acc_bcl = accuracy_score(y_test, bcl_pred)
acc_bcl

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_test, bcl_pred)

ax = sns.heatmap(cm, annot=True, cmap="Reds")

ax.set_xlabel('Prediksi')
ax.set_ylabel('Aktual')
plt.title('Confussion Matrix of Decision Bagging Classifier')
plt.show()

In [ ]:
# make classification report
print(classification_report(y_test, bcl_pred))

# **Random Forest**

In [ ]:
# declare model random forest with best parameter from hyperparameter tuning
rf = RandomForestClassifier(n_estimators=100,
                            criterion='entropy',
                            max_features=0.5,
                            min_samples_leaf=10,
                            random_state=11,
                            n_jobs=1)

In [ ]:
# fit data train into model
rf.fit(X_train, y_train)

In [ ]:
# do prediction
rf_pred = rf.predict(X_test)

In [ ]:
# print accuracy score
acc_rf = accuracy_score(y_test, rf_pred)
acc_rf

In [ ]:
# print classification report
print(classification_report(y_test, rf_pred))

# **Test Oversampling using SMOTE**

In [ ]:
# using imblearn to stratified label
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [ ]:
df_plt = pd.DataFrame(y_res)

In [ ]:
# plot y after oversampling
sns.countplot(x='HeartDisease', data=df_plt, edgecolor='black')

In [ ]:
# see proportion of labels
sns.countplot(x='HeartDisease', data=df)

# 410 tidak menderita sakit jantung
# 508 menderita sakit jantung

In [ ]:
# split the data
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res, 
                                                    y_res, 
                                                    test_size=0.25, 
                                                    random_state=11)

In [ ]:
# look shape of data before oversampling
print(f'Count of X: {X.shape}\ncount of y: {y.shape}')

In [ ]:
# look shape of data after oversampling
print(f'Count of X: {X_res.shape}\ncount of y: {y_res.shape}')

In [ ]:
# do a prediction and check accuracy score
pr_pred_res=bcl.predict(X_test_res)
acc_ovsbcl = accuracy_score(y_test_res, pr_pred_res)
acc_ovsbcl

In [ ]:
print(classification_report(y_test_res, pr_pred_res))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_test_res, pr_pred_res)

ax = sns.heatmap(cm, annot=True, cmap="Blues")

ax.set_xlabel('Prediksi')
ax.set_ylabel('Aktual')
plt.title('Confussion Matrix of Bagging Classifier Oversampling')
plt.figure(figsize=(4, 2))
plt.show()

# **Test With Scaled Data**

In [ ]:
ss = StandardScaler() # Standardization
mms = MinMaxScaler() # MinMaxScaler

In [ ]:
ss_X_train = ss.fit_transform(X_train)
ss_X_test = ss.fit_transform(X_test)
mms_X_train = mms.fit_transform(X_train)
mms_X_test = mms.fit_transform(X_test)

In [ ]:
# fitting the train data into model
dtc.fit(ss_X_train, y_train)

# do a prediction using test data
npr_pred1=dtc.predict(ss_X_test)
acc_ss_dtc = accuracy_score(y_test, npr_pred1)
print(f'accuration of Standarized Decision Tree : {acc_ss_dtc}')

# fitting the train data into model
dtc.fit(mms_X_train, y_train)

# do a prediction using test data
npr_pred2=dtc.predict(mms_X_test)
acc_mms_dtc = accuracy_score(y_test, npr_pred2)
print(f'accuration of MinMaxScaled Decision Tree : {acc_mms_dtc}')

In [ ]:
# fitting the train data into model
clf.fit(ss_X_train, y_train)

# do a prediction using test data
pr_pred1=clf.predict(ss_X_test)
acc_ss_dtcpr = accuracy_score(y_test, pr_pred1)
print(f'accuration of Standarized Decision Tree Pruning: {acc_ss_dtcpr}')

# fitting the train data into model
clf.fit(mms_X_train, y_train)

# do a prediction using test data
npr_pred2=clf.predict(mms_X_test)
acc_mms_dtcpr = accuracy_score(y_test, npr_pred2)
print(f'accuration of MinMaxScaled Decision Tree Pruning: {acc_mms_dtcpr}')

In [ ]:
# fitting the train data into model
bcl.fit(ss_X_train, y_train)

# do a prediction using test data
bcl_pred1=bcl.predict(ss_X_test)
acc_ss_bcl = accuracy_score(y_test, bcl_pred1)
print(f'accuration of Standarized Bagging Classifier: {acc_ss_bcl}')

# fitting the train data into model
bcl.fit(mms_X_train, y_train)

# do a prediction using test data
npr_pred2=bcl.predict(mms_X_test)
acc_mms_bcl = accuracy_score(y_test, npr_pred2)
print(f'accuration of MinMaxScaled Bagging Classifier: {acc_mms_bcl}')

In [ ]:
# fitting the train data into model
rf.fit(ss_X_train, y_train)

# do a prediction using test data
rf_pred1=rf.predict(ss_X_test)
acc_ss_rf = accuracy_score(y_test, rf_pred1)
print(f'accuration of Standarized Random Forest: {acc_ss_rf}')

# fitting the train data into model
rf.fit(mms_X_train, y_train)

# do a prediction using test data
rf_pred2=rf.predict(mms_X_test)
acc_mms_rf = accuracy_score(y_test, npr_pred2)
print(f'accuration of MinMaxScaled Random Forest: {acc_mms_rf}')

# **Test Splitting Using K-Fold Cross Validation**

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(clf, X, y, cv=5))
print(cross_val_score(clf, X, y, cv=5).mean())

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

Xcv = df.iloc[:, :-1]
ycv = df.iloc[:, -1]

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
    X_train_cv, X_test_cv = Xcv.iloc[train_index], Xcv.iloc[test_index]
    y_train_cv, y_test_cv = ycv.iloc[train_index], ycv.iloc[test_index]

In [ ]:
# KFold Cross Validation Decision Tree

# fitting the model
dtc.fit(X_train_cv, y_train_cv)
# do prediction with data test
dtccv_pred = dtc.predict(X_test_cv)
# write accuracy score
acc_dtccv = accuracy_score(y_test_cv, dtccv_pred)
acc_dtccv

In [ ]:
# KFold Cross Validation Decision Tree Pruning

# fitting the model
clf.fit(X_train_cv, y_train_cv)
# do prediction with data test
prdtccv_pred = clf.predict(X_test_cv)
# write accuracy score
acc_prdtccv = accuracy_score(y_test_cv, prdtccv_pred)
acc_prdtccv

In [ ]:
# KFold Cross Validation Random Forest

# fitting the model
rf.fit(X_train_cv, y_train_cv)
# do prediction with data test
rfcv_pred = rf.predict(X_test_cv)
# write accuracy score
acc_rfcv = accuracy_score(y_test_cv, rfcv_pred)
acc_rfcv

In [ ]:
# KFold Cross Validation Bagging Classifier

# fitting the model
bcl.fit(X_train_cv, y_train_cv)
# do prediction with data test
bclcv_pred = bcl.predict(X_test_cv)
# write accuracy score
acc_bclcv = accuracy_score(y_test_cv, bclcv_pred)
acc_bclcv

# **Test Using Repeat Stratified K-Fold**

In [ ]:
def model(classifier):
    classifier.fit(X_train,y_train)
    ml_pred = classifier.predict(X_test)
    cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
    print(classification_report(y_test, ml_pred))

In [ ]:
model(dtc)

In [ ]:
model(clf)

In [ ]:
model(bcl)

In [ ]:
model(rf)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

knn = KNN(n_neighbors=1)
model(knn)

In [ ]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

bnb = BernoulliNB()
model(bnb)

In [ ]:
gnb = GaussianNB()
model(gnb)

# **Combine Model With All Best Result**

In [ ]:
ss_X_train_res = ss.fit_transform(X_train_res)
ss_X_test_res = ss.fit_transform(X_test_res)

In [ ]:
# combine all best result in this case all best result improving model is {'standarization bcl', 'oversampling bcl'}
# fitting the train data into model
bcl.fit(ss_X_train_res, y_train_res)

# do a prediction using test data
ssbclres_pred=bcl.predict(ss_X_test_res)
acc_ssbclres = accuracy_score(y_test_res, ssbclres_pred)
print(f'accuration of Standarized Bagging Classifier: {acc_ssbclres}')

# **Model Comparison**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

data = {
    'Decision Tree': acc_dtc * 100,
    'Decision Tree Pruning': acc_prdtc * 100,
    'Bagging Classifier': acc_bcl * 100,
    'Random Forest': acc_rf * 100,
    'Bagging Classifier Oversampling': acc_ovsbcl * 100,
    'Decision Tree Standarization': acc_ss_dtc * 100,
    'Decision Tree MinMaxScaler': acc_mms_dtc * 100,
    'Decision Tree Pruning Standarization': acc_ss_dtcpr * 100,
    'Decision Tree Pruning MinMaxScaler': acc_mms_dtcpr * 100,
    'Bagging Classifier Standarization': acc_ss_bcl * 100,
    'Bagging Classifier MinMaxScaler': acc_mms_bcl * 100,
    'Random Forest Standarization': acc_ss_rf * 100,
    'Random Forest MinMaxScaler': acc_mms_rf * 100,
    'Bagging Classifier Oversampling': acc_ovsbcl * 100,
    'Bagging Classifier Standarization Oversampling': acc_ssbclres * 100
}

# Sort the dictionary by values in descending order
data = dict(sorted(data.items(), key=lambda x: x[1], reverse=True))

models = list(data.keys())
score = [round(val, 2) for val in data.values()]

fig = plt.figure(figsize=(10, 8))
ax = sns.barplot(x=score, y=models)

# Add rounded accuracy scores on the left side of each bar
for i in range(len(score)):
    ax.text(score[i] - 5, i, str(score[i]), ha='right', va='center', color='black', fontweight='light')

plt.xlabel("Score", size=16)
plt.xticks(size=12)
plt.ylabel("Models Used", size=16)
plt.yticks(size=12)
plt.title("Score for Different Models", size=20)
plt.show()